In [11]:
import os
import time
import json

import pandas as pd
import matplotlib.pyplot as plt
import pickle

from PIL import Image
from IPython.display import clear_output


In [3]:
# Leer el pickle. En su defecto crear otro dataframe 

pickle_path = '/Users/tonivila/Downloads/para_limpiar.pkl' # Especificar la URL
df = pd.read_pickle(pickle_path)
df.head()

,photo_id,label
30041,8cABbLSBC9RFqHSx_wgG5g,food
85008,Q8RtRWNR30z1U-st1f618Q,food
33552,9jeWcSmt2trLgGthSZjdwg,food
77935,Ns6QABZamxkHKbbWV3riwA,food
183677,ujxNd-z5iSJ1iR9Tw-E1vg,food


In [4]:
df.tail()

,photo_id,label
196365,yqNFHflhf-y9ZW2xtucy9A,menu
196567,yuDiGILa39ZN5DXZnElfig,menu
196872,yzotdcgH1Cb2TKwdwkwKaA,menu
198559,zXbjfZTuVfuqCBP2XKQlHg,menu
198766,zafQ51PjbkDaLn4k6gjbLg,menu


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21678 entries, 30041 to 198766
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   photo_id  21678 non-null  object
 1   label     21678 non-null  object
dtypes: object(2)
memory usage: 508.1+ KB


In [12]:
# Image Classifier

label = ['food', 'inside', 'outside', 'drink', 'menu', 'KO']
range_ = list(range(1, 7))
label_dict = dict(zip(range_, label))
cat_dict = dict(zip(label, range_))


def check_labels(category, img_folder, check_folder, start, end, checkpoint_number, dataframe):

    ids_dict = {}
    ids = list(dataframe[dataframe.label == category].photo_id)[start:end]
    len_ = len(ids)
    checkpoint = checkpoint_number

    # Loop por los photo_id de la categoría definida. Visualización y clasificación.
    for i in ids:
        current_label = list(dataframe[dataframe.photo_id == i].label)[0]

        # Control en el caso de que haya una imagen corrupta y no se pueda cargar. 
        while True:
            try:
                im = Image.open(img_folder + '/' + i + '.jpg')
                break

            except:
                ids_dict[i] = [current_label, 'Corrupt', 'KO']
                continue
        

        plt.title(f'{current_label}. (Quedan {len_} fotos)')
        plt.imshow(im)
        plt.show()
        
        print('¿Está bien tageada?')
        print('Sí - pulsa intro, 1: food, 2: inside, 3: outside, 4: drink, 5: menu, 6: KO')

        # Check si el valor está permitido y asignar las categorías.
        while True:

            # Guardar el diccionario en un json por seguridad
            if checkpoint == 0:
                json_name = f'checkpoin_cat_{category}.json'
                with open(check_folder + '/' + json_name, 'w') as save_check:
                    json.dump(ids_dict, save_check)
                
                print('Atención. Se ha grabado un checkpoint :)')
                checkpoint = checkpoint_number
                
            
            # Pedir opciones
            usr_input = input()
            
            if usr_input == '':
                ids_dict[i] = [current_label, current_label, 'OK']
                checkpoint -= 1
                len_ -= 1
                break

            else:
                try:
                    if int(usr_input) == cat_dict[category]:
                        print('Has tageado la misma categoría. Selecciona otra :)')
                        continue

                    elif int(usr_input) in list(label_dict.keys()):
                        ids_dict[i] = [current_label, label_dict[int(usr_input)], 'KO']
                        checkpoint -= 1
                        len_ -= 1
                        break
                    else:
                        print('Valor equivocado :( Las opciones van del 1 al 6. Intenta otra vez.')
                        continue

                except:
                    print('Valor equivocado :( No es un número. Intenta otra vez.')

        clear_output(wait=True)


    return ids_dict


In [13]:
# Categorías ['food', 'inside', 'outside', 'drink', 'menu']
# Parámetros: def check_labels(category, img_folder, check_folder, start, end, checkpoint_number, dataframe):

"""
Temas pendientes: 
- Ver si tiene sentido tener un start / stop o incluimos directamente todo el dataset
- Checkpoint_number - Ver cúal es el número óptiom de guardados. Realmente es bastante rápido
por lo que puede guardar con bastante frecuencia (en el ejemplo esta puesto 2 para ver que funciona)
"""

image_path = '/Users/tonivila/Downloads/TheValley_TFM/imagenes_negocios'    # Ruta con las imagenes
save_path = '/Users/tonivila/Downloads'                                     # Ruta donde queremos que se guade el json


print(check_labels('food',              # Categoria
                    image_path,         # ruta imagenes
                    save_path,          # ruta guardado checkpoint
                    0,                  # donde inicia el dataframe (por posición)
                    10,                 # donde termina el dataframe (por posición)
                    2,                  # Frecuencia con la que guarda el checkpoint
                    df))                # Dataframe de referencia


{'8cABbLSBC9RFqHSx_wgG5g': ['food', 'food', 'OK'], 'Q8RtRWNR30z1U-st1f618Q': ['food', 'food', 'OK'], '9jeWcSmt2trLgGthSZjdwg': ['food', 'food', 'OK'], 'Ns6QABZamxkHKbbWV3riwA': ['food', 'food', 'OK'], 'ujxNd-z5iSJ1iR9Tw-E1vg': ['food', 'food', 'OK'], 'NYhaETJyFbCyvwgGKsj2BA': ['food', 'food', 'OK'], 'XCCuNpEQ-SiltHVCZ6jUwg': ['food', 'food', 'OK'], 'hQCbRgn_gdl_zCqa-UrlQQ': ['food', 'food', 'OK'], 'cWgobpio-YuoWOUeExUdWA': ['food', 'food', 'OK'], 'l3PH93TMO1XBHKHd2Wbzuw': ['food', 'food', 'OK']}
